# Experimenting with AHP Tree parsing

In [1]:
import pandas as pd
import re
import sys 
import os
sys.path.insert(0, os.path.abspath("../"))
from pyanp.ahptree import AHPTree

In [2]:
df = pd.read_excel("../tests/data/AHPTreeData.xlsx")

In [3]:
cols = df.columns.values

In [4]:
cols

array(['Age', 'Eyes', 'A vs B', 'B vs C', 'A vs C', 'A1 vs A2 wrt A',
       'A1 vs A3 wrt A', 'A2 vs A3 wrt A', 'B1 vs B2 wrt B',
       'C1 vs C2 wrt C', 'Alt1 wrt A1', 'Alt1 wrt A2', 'Alt1 wrt A3',
       'Alt1 wrt B1', 'Alt1 wrt B2', 'Alt1 wrt C1', 'Alt1 wrt C2',
       'Alt2 wrt A1', 'Alt2 wrt A2', 'Alt2 wrt A3', 'Alt2 wrt B1',
       'Alt2 wrt B2', 'Alt2 wrt C1', 'Alt2 wrt C2', 'Alt3 wrt A1',
       'Alt3 wrt A2', 'Alt3 wrt A3', 'Alt3 wrt B1', 'Alt3 wrt B2',
       'Alt3 wrt C1', 'Alt3 wrt C2'], dtype=object)

In [5]:
__wrtre = re.compile("^(.+)\s+vs\s+(.+)\s+wrt\s+(.+)$")
__avsb = re.compile("^(.+)\s+vs\s+(.+)$")
__directre = re.compile("^(.+)\s+wrt\s+(.+)$")
def col_parse(col, theGoalNode="Goal"):
    #print("For col="+col)
    minfo = __wrtre.match(col)
    dom, rec, wrt = [None]*3
    if minfo is not None:
        dom, rec, wrt = minfo.groups()
        return "pw", dom, rec, wrt
    else:
        # We do not have wrt, try a vs b
        minfo = __avsb.match(col)
        if minfo is not None:
            wrt = theGoalNode
            dom, rec = minfo.groups() 
            return "pw", dom, rec, wrt
    #If we made it here, it is not pairwise, try direct
    minfo = __directre.match(col)
    if minfo is not None:
        node, wrt = minfo.groups()
        return "direct", node, wrt
    # If we made it here, it has to be demographic
    return "demo", col

def info_ispw(info):
    return info[0] == "pw"

def info_isdirect(info):
    return info[0] == "direct"

def info_wrt(info):
    if info_ispw(info) or info_isdirect(info):
        return info[-1]
    else:
        return None

def info_compares(info, node):
    if info_ispw(info):
        return (info[1]==node) or (info[2] == node)
    elif info_isdirect(info):
        return (info[1]==node)
    else:
        return False
    
def get_nodes_from_parsed(infos):
    nodes = []
    for info in infos:
        if info_ispw(info):
            nodes.append(info[1])
            nodes.append(info[2])
            nodes.append(info[3])
        elif info_isdirect(info):
            nodes.append(info[1])
            nodes.append(info[2])
    rval = list(dict.fromkeys(nodes))
    return rval

def node_parents(infos, node):
    rval = []
    for info in infos:
        if info_compares(info, node):
            rval.append(info_wrt(info))
    rval = list(dict.fromkeys(rval))
    return rval

def node_alts(infos, nodes):
    rval = [node for node in nodes if len(node_parents(infos, node))>1]
    return rval

def node_root(infos, nodes):
    rval = [node for node in nodes if len(node_parents(infos, node))<=0]
    return rval

def node_children(infos, node):
    rval = []
    for info in infos:
        if info_wrt(info) == node:
            if info_ispw(info):
                rval.append(info[1])
                rval.append(info[2])
            elif info_isdirect(info):
                rval.append(info[1])
    rval = list(dict.fromkeys(rval))
    return rval

def create_ahptree(infos, currentAHPTree=None, currentNode=None):
    nodes = get_nodes_from_parsed(infos)
    alts = node_alts(infos, nodes)
    print(alts)
    root = node_root(infos, nodes)
    if len(root)>1:
        raise ValueError("Too many root nodes, needs exactly1, had "+str(root))
    root = root[0]
    isToplevel = False
    if currentAHPTree is None:
        isToplevel=True
        currentAHPTree = AHPTree(root)
        currentNode = root
    for kid in node_children(infos, currentNode):
        if kid not in alts:
            print("Adding node="+kid+" under="+currentNode)
            currentAHPTree.add_child(kid, currentNode)
            create_ahptree(infos, currentAHPTree, kid)
    # Finally add alts, but only if in top-level
    if isToplevel:
        for alt in alts:
            currentAHPTree.add_alt(alt)
    return currentAHPTree

In [6]:

infos = [col_parse(col) for col in cols]
nodes = get_nodes_from_parsed(infos)
parents = {node:node_parents(infos, node) for node in nodes}
alts = node_alts(infos, nodes)
root = node_root(infos, nodes)
if len(root)>1:
    raise ValueError("Too many root nodes, needs exactly 1, had "+str(root))
root = root[0]
tree = AHPTree(root_name=root)


In [7]:
tree=create_ahptree(infos)

['Alt1', 'Alt2', 'Alt3']
Adding node=A under=Goal
['Alt1', 'Alt2', 'Alt3']
Adding node=A1 under=A
['Alt1', 'Alt2', 'Alt3']
Adding node=A2 under=A
['Alt1', 'Alt2', 'Alt3']
Adding node=A3 under=A
['Alt1', 'Alt2', 'Alt3']
Adding node=B under=Goal
['Alt1', 'Alt2', 'Alt3']
Adding node=B1 under=B
['Alt1', 'Alt2', 'Alt3']
Adding node=B2 under=B
['Alt1', 'Alt2', 'Alt3']
Adding node=C under=Goal
['Alt1', 'Alt2', 'Alt3']
Adding node=C1 under=C
['Alt1', 'Alt2', 'Alt3']
Adding node=C2 under=C
['Alt1', 'Alt2', 'Alt3']


In [8]:
tree.alt_names

['Alt1', 'Alt2', 'Alt3']

In [9]:
root

'Goal'